# A5 小書產生器（gTTS 影片強化版｜Voila 可下載）
- 不改變照片長寬比（自動置中留白）。
- 字幕／旁白會忽略 `[...]` 內容。
- 每張時間不設上/下限，依字數×秒數估算（預設 0.3s/字）。
- 人聲類型：常用語系 preset 或自訂。
- 封面/封底固定不加頁碼；內頁可勾選是否加頁碼（-1-）。

In [ ]:
from pypinyin import lazy_pinyin, Style
import re, ipywidgets as widgets
from IPython.display import display

p_pinyin_on = widgets.Checkbox(value=False, description='顯示拼音（文字後加括號）')
p_pinyin_style = widgets.Dropdown(options=[('帶音調 mā','tone'),('數字聲調 ma1','tone_num'),('無聲調 ma','plain')], value='tone', description='拼音格式')
display(p_pinyin_on, p_pinyin_style)

_bracket_pat = re.compile(r'\[[^\]]*\]')
_space_pat = re.compile(r'\s+')
def _clean_for_pinyin(s: str) -> str:
    s = _bracket_pat.sub('', s)
    s = _space_pat.sub(' ', s).strip()
    return s

def to_pinyin_line(s: str, style: str='tone') -> str:
    if not s: return s
    s = _clean_for_pinyin(s)
    if style == 'tone':
        parts = lazy_pinyin(s, style=Style.TONE3, errors='ignore')
        return ' '.join(parts)
    elif style == 'tone_num':
        parts = lazy_pinyin(s, style=Style.TONE3, errors='ignore')
        return ' '.join(parts)
    else:  # plain
        parts = lazy_pinyin(s, style=Style.NORMAL, errors='ignore')
        return ' '.join(parts)

def apply_pinyin_if_enabled(text: str) -> str:
    if not p_pinyin_on.value:
        return _clean_for_pinyin(text)
    py = to_pinyin_line(text, style=p_pinyin_style.value)
    return f"{_clean_for_pinyin(text)} ({py})"


In [ ]:
import sys, subprocess, os, shutil, importlib
import ipywidgets as widgets
from IPython.display import display

def _pip_install(pkgs):
    print('→ pip install', ' '.join(pkgs))
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs)

def _ensure_av_deps():
    need = []
    try: importlib.import_module('gtts')
    except Exception: need.append('gTTS==2.5.1')
    try: importlib.import_module('edge_tts')
    except Exception: need.append('edge-tts==7.2.3')
    try:
        importlib.import_module('moviepy')
        importlib.import_module('imageio')
    except Exception:
        need += ['moviepy==1.0.3','imageio==2.35.1']
    if need: _pip_install(need)

    ffmpeg_path = shutil.which('ffmpeg')
    if not ffmpeg_path:
        try:
            import imageio_ffmpeg
        except Exception:
            _pip_install(['imageio-ffmpeg'])
            import imageio_ffmpeg
        ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
    os.environ['IMAGEIO_FFMPEG_EXE'] = ffmpeg_path
    print('✅ 影音套件已就緒, ffmpeg=', ffmpeg_path)

p_make_av = widgets.Checkbox(value=False, description='製作影音 (勾選自動安裝套件)')
out = widgets.Output()

def _on_toggle(change):
    if change['name']=='value' and change['new']:
        with out:
            print('開始安裝影音所需套件…')
            _ensure_av_deps()
            _enable_av_ui()
            print('— 完成 —\n')


# 啟用相關 UI（若這些按鈕/控制存在的話，就解除鎖定）
def _enable_av_ui():
    import builtins
    g = globals()
    names = list(g.keys())
    candidates = [
        'btn_render_video','btn_make_video','btn_export_video',
        'btn_tts','btn_make_audio','btn_export_audio',
        'btn_run','btn_generate','btn_build','btn_save'
    ]
    enabled = []
    for name in candidates:
        obj = g.get(name)
        try:
            import ipywidgets as _w
            if isinstance(obj, _w.Button):
                obj.disabled = False
                enabled.append(name)
        except Exception:
            pass
    try:
        from IPython.display import display
        import ipywidgets as _w
        msg = _w.HTML(value=f"<b>影音環境就緒</b>：已啟用 {', '.join(enabled) if enabled else '（未偵測到相關按鈕）'}")
        display(msg)
    except Exception:
        print('影音環境就緒。')

p_make_av.observe(_on_toggle, names='value')
display(p_make_av, out)


In [ ]:
import io, os, re, base64, urllib.request, time, math, tempfile
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# pypinyin（可選）
try:
    from pypinyin import pinyin as pinyin_fn, Style
    HAVE_PINYIN=True
except Exception:
    HAVE_PINYIN=False
    pinyin_fn=None
    class _S: pass
    Style=_S()

from docx import Document
from docx.shared import Mm, Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

from PIL import Image, ImageDraw, ImageFont, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

try:
    from moviepy.editor import ImageClip, AudioFileClip, CompositeAudioClip, concatenate_videoclips
    import numpy as np
    MOVIEPY_OK=True; MOVIEPY_ERR=''
except Exception as e:
    MOVIEPY_OK=False; MOVIEPY_ERR=str(e)

try:
    from gtts import gTTS
    HAVE_GTTS=True
except Exception:
    HAVE_GTTS=False

try:
    from ipywidgets import FileDownload
    HAVE_FD=True
except Exception:
    HAVE_FD=False


In [ ]:
# --------- 工具 ---------
FWP='％'

def ensure_rgb_jpeg(img_bytes, quality=92):
    im = Image.open(io.BytesIO(img_bytes))
    if im.mode not in ('RGB','L'): im = im.convert('RGB')
    out = io.BytesIO(); im.save(out, format='JPEG', quality=quality)
    return out.getvalue()

def parse_and_strip_size_anywhere(text, default_pt=12):
    m=re.search(r"\[(\d+)pt\]", text)
    if m: return int(m.group(1)), text[:m.start()]+text[m.end():]
    return default_pt, text

def tokenize_line(line):
    pattern=re.compile(r"\[\s*左圖\s*(\d+)?\s*[%"+FWP+"]?\s*\]\s*\[\s*右圖\s*(\d+)?\s*[%"+FWP+"]?\s*\]|\[\s*圖片\s*(\d+)?\s*[%"+FWP+"]?\s*\]")
    tokens=[]; i=0
    while True:
        m=pattern.search(line,i)
        if not m:
            tokens.append(('text', line[i:])); break
        if m.start()>i: tokens.append(('text', line[i:m.start()]))
        if m.group(1) is not None or m.group(2) is not None:
            tokens.append(('double', int(m.group(1)) if m.group(1) else None, int(m.group(2)) if m.group(2) else None))
        else:
            tokens.append(('single', int(m.group(3)) if m.group(3) else None))
        i=m.end()
    return tokens

def set_section_to_a5(section):
    section.page_width=Mm(148); section.page_height=Mm(210)
    section.left_margin=Mm(15); section.right_margin=Mm(15)
    section.top_margin=Mm(15); section.bottom_margin=Mm(15)

def set_page_number_start(section, start_at=1):
    sectPr=section._sectPr; pg=OxmlElement('w:pgNumType'); pg.set(qn('w:start'), str(int(start_at)))
    for el in list(sectPr):
        if el.tag==pg.tag: sectPr.remove(el)
    sectPr.append(pg)

def add_text_paragraph(doc, text, size_pt, center=False):
    text=text.strip();
    if not text: return None
    p=doc.add_paragraph(text); p.alignment=WD_ALIGN_PARAGRAPH.CENTER if center else WD_ALIGN_PARAGRAPH.JUSTIFY
    for r in p.runs: r.font.size=Pt(size_pt)
    return p

def add_single_image_paragraph(doc, img_bytes, width_pct=None, center=True):
    page_w_in=(148-15-15)/25.4
    width_in=page_w_in if width_pct is None else max(1.0, min(page_w_in, page_w_in*(width_pct/100.0)))
    p=doc.add_paragraph(); p.alignment=WD_ALIGN_PARAGRAPH.CENTER if center else WD_ALIGN_PARAGRAPH.JUSTIFY
    p.add_run().add_picture(io.BytesIO(ensure_rgb_jpeg(img_bytes)), width=Inches(width_in))

def add_double_image_table(doc, left_bytes, right_bytes, l_pct, r_pct):
    page_w_in=(148-15-15)/25.4; padding=0.2; base=(page_w_in-padding)/2.0
    l_in=base if l_pct is None else max(0.8, min(base, base*(l_pct/100.0)))
    r_in=base if r_pct is None else max(0.8, min(base, base*(r_pct/100.0)))
    table=doc.add_table(rows=1, cols=2)
    for ci,b,w in [(0,left_bytes,l_in),(1,right_bytes,r_in)]:
        para=table.rows[0].cells[ci].paragraphs[0]; para.alignment=WD_ALIGN_PARAGRAPH.CENTER
        para.add_run().add_picture(io.BytesIO(ensure_rgb_jpeg(b)), width=Inches(w))


In [ ]:
# --------- 拼音與字幕前處理 ---------
def strip_bracket_tags(s:str)->str:
    return re.sub(r"\[.*?\]", "", s)

def charwise_pinyin(text:str)->str:
    if not HAVE_PINYIN: return text
    out=[]
    for ch in text:
        if '\u4e00'<=ch<='\u9fff':
            py=pinyin_fn(ch, style=Style.TONE, strict=False)
            py_s=py[0][0] if py and py[0] else ''
            out.append(f"{ch}({py_s})")
        else: out.append(ch)
    return ''.join(out)

def sentencewise_pinyin(text:str)->str:
    if not HAVE_PINYIN: return text
    chinese=[ch for ch in text if '\u4e00'<=ch<='\u9fff']
    if not chinese: return text
    py_list=pinyin_fn(''.join(chinese), style=Style.TONE, strict=False)
    py_join=' '.join([itm[0] for itm in py_list if itm and itm[0]])
    return f"{text}（{py_join}）"


In [ ]:
# --------- UI ---------
def image_manager(title:str):
    uploader=widgets.FileUpload(accept='image/*', multiple=True, description=f'上傳{title}')
    lst=widgets.Select(options=[], rows=6, description='順序')
    btn_up=widgets.Button(description='上移'); btn_down=widgets.Button(description='下移')
    btn_del=widgets.Button(description='刪除'); btn_clear=widgets.Button(description='清空')
    add_box=widgets.Textarea(placeholder='貼 URL 或 data:image/...;base64,... 或純 base64（每行一張）', layout=widgets.Layout(height='90px', width='100%'))
    auto_add=widgets.Checkbox(value=True, description='自動解析貼上')
    add_btn=widgets.Button(description='新增到清單', button_style='info')
    status=widgets.HTML(''); images=[]
    def refresh():
        lst.options=[f"{i+1:02d}. {it['name']} ({len(it['bytes'])//1024}KB)" for i,it in enumerate(images)]; lst.index=0 if images else None
    def on_upload(change):
        for meta in list(uploader.value):
            content=meta.get('content', b''); name=meta.get('name','image')
            if content: images.append({'name':name,'bytes':content})
        refresh(); status.value=f"<span style='color:green'>已加入 {len(images)} 張</span>"
    uploader.observe(on_upload, names='value')
    def move(d):
        if not images or lst.index is None: return
        i=lst.index; j=i+d
        if 0<=j<len(images): images[i],images[j]=images[j],images[i]; refresh(); lst.index=j
    btn_up.on_click(lambda _: move(-1)); btn_down.on_click(lambda _: move(1))
    btn_del.on_click(lambda _: (images.pop(lst.index), refresh()) if (images and lst.index is not None) else None)
    btn_clear.on_click(lambda _: (images.clear(), refresh()))
    def _try_parse_line(ln):
        try:
            if ln.startswith('data:image'): return base64.b64decode(ln.split(',',1)[1])
            if ln.startswith('http://') or ln.startswith('https://'):
                with urllib.request.urlopen(ln) as r: return r.read()
            return base64.b64decode(ln, validate=True)
        except Exception: return None
    def do_add(_=None,*,auto=False):
        lines=[ln.strip() for ln in add_box.value.splitlines() if ln.strip()]; add=0; fail=0
        for ln in lines:
            b=_try_parse_line(ln)
            if b: images.append({'name':f'added_{len(images)+1}.jpg','bytes':b}); add+=1
            else: fail+=1
        if add: refresh()
        if not auto: add_box.value=''
        if add or fail:
            status.value=f"<span style='color:{'green' if add and not fail else '#a60' if add else '#a00'}'>成功 {add}；失敗 {fail}</span>"
    add_btn.on_click(lambda _: do_add())
    add_box.observe(lambda ch: do_add(auto=True) if (auto_add.value and '\n' in (ch['new'] or '')) else None, names='value')
    return widgets.VBox([uploader, widgets.HBox([lst, widgets.VBox([btn_up, btn_down, btn_del, btn_clear])]), widgets.VBox([add_box, widgets.HBox([add_btn, auto_add])]), status]), images

warns=[]
if not MOVIEPY_OK: warns.append(f"⚠️ moviepy 不可用：{MOVIEPY_ERR}")
if not HAVE_PINYIN: warns.append("⚠️ pypinyin 未安裝，拼音功能停用。")
if not HAVE_GTTS: warns.append("⚠️ gTTS 未安裝，旁白功能停用。")
banner=widgets.HTML('<br>'.join(warns)) if warns else widgets.HTML('')

txt_cover=widgets.Textarea(value='神奇的迴力鏢[18pt]\n[圖片50%]\n種是希望', description='封面文字', layout=widgets.Layout(width='100%', height='140px'))
txt_body =widgets.Textarea(value='第一段文字。\n\n第二段文字。\n\n第三段文字。', description='內頁文字', layout=widgets.Layout(width='100%', height='180px'))
txt_back =widgets.Textarea(value='封底簡介……\n[左圖60%][右圖60%]', description='封底文字', layout=widgets.Layout(width='100%', height='140px'))

cover_default_pct=widgets.BoundedIntText(value=100, min=10, max=100, step=1, description='封面預設%')
body_default_pct =widgets.BoundedIntText(value=100, min=10, max=100, step=1, description='內頁預設%')
back_default_pct =widgets.BoundedIntText(value=100, min=10, max=100, step=1, description='封底預設%')

chk_body_pagenum=widgets.Checkbox(value=True, description='內頁加頁碼 (-1-)')
chk_pagebreak =widgets.Checkbox(value=False, description='每段文字自動換頁')
dropdown_pinyin=widgets.Dropdown(options=['不加拼音','字後拼音','整句拼音'], value='不加拼音', description='拼音模式')

chk_make_video =widgets.Checkbox(value=False, description='是否產生影片 (mp4)')
chk_video_cover=widgets.Checkbox(value=True, description='將封面加入影片')
chk_video_back =widgets.Checkbox(value=True, description='將封底加入影片')
vid_w=widgets.BoundedIntText(value=1280, min=640, max=3840, step=10, description='寬(px)')
vid_h=widgets.BoundedIntText(value=720,  min=360, max=2160, step=10, description='高(px)')
vid_fps=widgets.BoundedIntText(value=24, min=10, max=60, step=1, description='FPS')
sec_per_char=widgets.FloatText(value=0.35, description='每字秒數(估)')
fade_dur=widgets.FloatText(value=0.5, description='淡入/淡出秒數')

voice_type=widgets.Dropdown(options=['女聲(台灣風格)','女聲(大陸普通話)','女聲(香港風格)','英語女聲','自訂(下方語系/地區)'], value='女聲(台灣風格)', description='人聲類型')
voice_lang=widgets.Dropdown(options=['zh-TW','zh-CN','en'], value='zh-TW', description='語音語系')
voice_tld =widgets.Dropdown(options=['com','com.hk','com.tw'], value='com.tw', description='語音地區')
voice_slow=widgets.Checkbox(value=False, description='慢速')

def apply_voice_preset(_=None):
    p=voice_type.value
    if p=='女聲(台灣風格)': voice_lang.value='zh-TW'; voice_tld.value='com.tw'; voice_slow.value=False
    elif p=='女聲(大陸普通話)': voice_lang.value='zh-CN'; voice_tld.value='com'; voice_slow.value=False
    elif p=='女聲(香港風格)': voice_lang.value='zh-CN'; voice_tld.value='com.hk'; voice_slow.value=False
    elif p=='英語女聲': voice_lang.value='en'; voice_tld.value='com'; voice_slow.value=False
voice_type.observe(apply_voice_preset, names='value'); apply_voice_preset()

ui_cover, imgs_cover=image_manager('封面圖片（可多張）')
ui_body,  imgs_body =image_manager('內頁圖片（可多張）')
ui_back,  imgs_back =image_manager('封底圖片（可多張）')

btn_doc=widgets.Button(description='生成 A5 小書（docx）', button_style='success')
btn_vid=widgets.Button(description='生成影片（mp4）', button_style='primary')
out=widgets.Output()

display(widgets.VBox([
    banner,
    widgets.HBox([widgets.VBox([txt_cover, cover_default_pct]), widgets.VBox([dropdown_pinyin, chk_pagebreak])]),
    ui_cover,
    widgets.Label('—— 內頁 ——'),
    txt_body,
    widgets.HBox([body_default_pct, chk_body_pagenum]),
    ui_body,
    widgets.Label('—— 封底 ——'),
    txt_back,
    back_default_pct,
    ui_back,
    widgets.Label('—— 影片 ——'),
    widgets.HBox([chk_make_video, chk_video_cover, chk_video_back]),
    widgets.HBox([vid_w, vid_h, vid_fps, sec_per_char, fade_dur]),
    widgets.HBox([voice_type, voice_lang, voice_tld, voice_slow]),
    widgets.HBox([btn_doc, btn_vid]),
    out
]))


In [ ]:
# --------- DOCX ---------
def emit_stream(doc, text, size_pt, imgs, default_pct, center=False):
    tokens=tokenize_line(text)
    for tk in tokens:
        if tk[0]=='text': add_text_paragraph(doc, tk[1], size_pt, center=center)
        elif tk[0]=='single' and imgs:
            pct=tk[1] if tk[1] is not None else (default_pct or 100)
            add_single_image_paragraph(doc, imgs.pop(0), width_pct=pct, center=True)
        elif tk[0]=='double' and len(imgs)>=2:
            lp, rp=tk[1], tk[2]; add_double_image_table(doc, imgs.pop(0), imgs.pop(0), lp, rp)

def build_docx_bytes():
    cover_text=txt_cover.value; body_text=txt_body.value; back_text=txt_back.value
    cover_imgs=[it['bytes'] for it in imgs_cover]
    body_imgs=[it['bytes'] for it in imgs_body]
    back_imgs=[it['bytes'] for it in imgs_back]
    cover_pct=int(cover_default_pct.value or 100); body_pct=int(body_default_pct.value or 100); back_pct=int(back_default_pct.value or 100)

    doc=Document(); set_section_to_a5(doc.sections[0])
    sz, txt=parse_and_strip_size_anywhere(cover_text, default_pt=18)
    emit_stream(doc, txt, sz, cover_imgs, cover_pct, center=True)
    if cover_imgs: add_single_image_paragraph(doc, cover_imgs.pop(0), width_pct=cover_pct, center=True)

    doc.add_section(); set_section_to_a5(doc.sections[-1]); set_page_number_start(doc.sections[-1], 1)
    if chk_body_pagenum.value:
        footer=doc.sections[-1].footer
        p=footer.paragraphs[0] if footer.paragraphs else footer.add_paragraph(); p.alignment=WD_ALIGN_PARAGRAPH.CENTER
        p.add_run('-'); fb=OxmlElement('w:fldChar'); fb.set(qn('w:fldCharType'),'begin')
        instr=OxmlElement('w:instrText'); instr.text=' PAGE '
        fe=OxmlElement('w:fldChar'); fe.set(qn('w:fldCharType'),'end')
        r=OxmlElement('w:r'); r.append(fb); r.append(instr); r.append(fe); p._p.append(r); p.add_run('-')

    lines=body_text.split('\n'); non_empty=[ln for ln in lines if ln.strip()!='']
    total=len(non_empty); done=0
    for raw in lines:
        line=raw.strip()
        if line=='': doc.add_paragraph(''); continue
        size_pt, text=parse_and_strip_size_anywhere(line, default_pt=12)
        tokens=tokenize_line(text); has_tag=any(tk[0]!='text' for tk in tokens)
        emit_stream(doc, text, size_pt, body_imgs, body_pct, center=False)
        if (not has_tag) and body_imgs:
            add_single_image_paragraph(doc, body_imgs.pop(0), width_pct=body_pct, center=True)
        done+=1
        if chk_pagebreak.value and done<total: doc.add_page_break()

    doc.add_section(); set_section_to_a5(doc.sections[-1])
    sz2, txt2=parse_and_strip_size_anywhere(back_text, default_pt=12)
    emit_stream(doc, txt2, sz2, back_imgs, back_pct, center=True)
    if back_imgs: add_single_image_paragraph(doc, back_imgs.pop(0), width_pct=back_pct, center=True)

    buf=io.BytesIO(); doc.save(buf); data=buf.getvalue(); _=Document(io.BytesIO(data))
    return data


In [ ]:
# --------- 影片：維持比例 / 忽略[] / gTTS ---------
def pick_cjk_font(size_px):
    for p in ['/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc',
              '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',
              '/usr/share/fonts/truetype/noto/NotoSansCJK.ttc',
              '/usr/share/fonts/truetype/arphic/ukai.ttc',
              '/usr/share/fonts/truetype/arphic/uming.ttc']:
        if os.path.exists(p):
            try: return ImageFont.truetype(p, size=size_px)
            except Exception: pass
    return ImageFont.load_default()

def text_size(draw, s, font):
    l,t,r,b=draw.textbbox((0,0), s, font=font); return r-l, b-t

def wrap_text_by_width(draw, text, font, max_width):
    lines=[]; cur=''
    for ch in text:
        w,_=text_size(draw, cur+ch, font)
        if w<=max_width: cur+=ch
        else:
            if cur: lines.append(cur)
            cur=ch
    if cur: lines.append(cur)
    return lines

def strip_brackets_and_spaces(s:str)->str:
    s=re.sub(r"\[.*?\]","", s)
    return re.sub(r"\s+"," ", s).strip()

def burn_subtitle(img_rgba, text):
    text=strip_brackets_and_spaces(text)
    if not text: return img_rgba
    w,h=img_rgba.size; fs=max(14,int(28*h/720.0)); font=pick_cjk_font(fs)
    overlay=Image.new('RGBA',(w,h),(0,0,0,0)); draw=ImageDraw.Draw(overlay)
    margin=int(w*0.06); max_w=w-2*margin
    lines=wrap_text_by_width(draw, text, font, max_w)
    _,lh=text_size(draw,'測',font); lh=int(lh*1.3); total=lh*len(lines); y=h-total-int(h*0.08)
    for i,l in enumerate(lines):
        ww,_=text_size(draw,l,font); x=(w-ww)//2
        draw.text((x+2,y+2+i*lh), l, font=font, fill=(0,0,0,100))
        draw.text((x,y+i*lh), l, font=font, fill=(255,255,255,255))
    return Image.alpha_composite(img_rgba, overlay)

def fit_keep_ratio(img, W, H, bg=(255,255,255,255)):
    canvas=Image.new('RGBA',(W,H),bg)
    w,h=img.size; scale=min(W/w, H/h)
    nw,nh=max(1,int(w*scale)), max(1,int(h*scale))
    img2=img.resize((nw,nh))
    x=(W-nw)//2; y=(H-nh)//2
    canvas.paste(img2,(x,y))
    return canvas

def compose_frame(img_bytes, W, H, text_to_burn=None):
    base=Image.new('RGBA',(W,H),(255,255,255,255)) if img_bytes is None else fit_keep_ratio(Image.open(io.BytesIO(img_bytes)).convert('RGBA'), W, H)
    if text_to_burn: base=burn_subtitle(base, text_to_burn)
    buf=io.BytesIO(); base.convert('RGB').save(buf, format='JPEG', quality=95); return buf.getvalue()

def estimate_dur_by_text(text:str, sec_per_char:float=0.3):
    text=strip_brackets_and_spaces(text)
    n=max(1,len([c for c in text if not c.isspace()]))
    return float(max(0.01, sec_per_char)*n)

def tts_to_mp3(text:str, dst:str, lang='zh-TW', tld='com.tw', slow=False):
    if not HAVE_GTTS: return False
    try:
        text=strip_brackets_and_spaces(text)
        gTTS(text=text, lang=lang, tld=tld, slow=slow).save(dst)
        return os.path.exists(dst) and os.path.getsize(dst)>0
    except Exception:
        return False


In [ ]:
# --------- 事件處理（含三重下載） ---------
btn_doc, btn_vid

def apply_pinyin_mode(s:str)->str:
    mode=dropdown_pinyin.value
    if mode=='字後拼音' and HAVE_PINYIN: return charwise_pinyin(s)
    if mode=='整句拼音' and HAVE_PINYIN: return sentencewise_pinyin(s)
    return s

def resolve_voice_params():
    p=voice_type.value; lang=voice_lang.value; tld=voice_tld.value; slow=voice_slow.value
    if p!='自訂(下方語系/地區)':
        if p=='女聲(台灣風格)': lang,tld,slow='zh-TW','com.tw',False
        elif p=='女聲(大陸普通話)': lang,tld,slow='zh-CN','com',False
        elif p=='女聲(香港風格)': lang,tld,slow='zh-CN','com.hk',False
        elif p=='英語女聲': lang,tld,slow='en','com',False
    return lang,tld,slow

def on_click_doc(_):
    with out:
        clear_output()
        try:
            data=build_docx_bytes(); fname=f'a5_book_{int(time.time())}.docx'
            b64=base64.b64encode(data).decode()
            display(HTML('<div style="color:green;">DOCX 完成！</div>'))
            display(HTML(f'<a download="{fname}" href="data:application/vnd.openxmlformats-officedocument.wordprocessingml.document;base64,{b64}">📥 下載 {fname}</a>'))
        except Exception as e:
            display(HTML(f"<div style='color:red;'>失敗：{e}</div>"))

def on_click_video(_):
    with out:
        clear_output()
        if not chk_make_video.value:
            display(HTML('<div style="color:#a60;">未勾選「是否產生影片」，已略過。</div>')); return
        try:
            if not MOVIEPY_OK: raise RuntimeError('moviepy 尚不可用，請在 requirements.txt 加 moviepy 與 imageio-ffmpeg')
            os.makedirs('output', exist_ok=True)
            mp4=f'output/a5_book_{int(time.time())}.mp4'
            # 組片段
            cover_text=txt_cover.value.strip(); body_text=txt_body.value; back_text=txt_back.value.strip()
            cover_imgs=[it['bytes'] for it in imgs_cover]; body_imgs=[it['bytes'] for it in imgs_body]; back_imgs=[it['bytes'] for it in imgs_back]
            segs=[]
            if chk_video_cover.value and (cover_text or cover_imgs): segs.append((apply_pinyin_mode(cover_text), cover_imgs[0] if cover_imgs else None))
            lines=[ln.strip() for ln in body_text.split('\n') if ln.strip()!='']
            for i,line in enumerate(lines): segs.append((apply_pinyin_mode(line), body_imgs[i] if i<len(body_imgs) else None))
            if chk_video_back.value and (back_text or back_imgs): segs.append((apply_pinyin_mode(back_text), back_imgs[0] if back_imgs else None))

            # 生成 clip
            clips=[]; tmp=[]; lang,tld,slow=resolve_voice_params()
            for idx,(tx,img) in enumerate(segs):
                dur = float(max(0.01, sec_per_char.value or 0.3)) * max(1,len(strip_brackets_and_spaces(tx)))
                frame = compose_frame(img, int(vid_w.value), int(vid_h.value), text_to_burn=tx)
                arr = np.array(Image.open(io.BytesIO(frame)).convert('RGB'))
                c = ImageClip(arr).set_duration(dur).fadein(float(fade_dur.value or 0.5)).fadeout(float(fade_dur.value or 0.5))
                mp3=os.path.join(tempfile.gettempdir(), f'v_{int(time.time()*1000)}_{idx}.mp3')
                if tts_to_mp3(tx, mp3, lang=lang, tld=tld, slow=slow):
                    try:
                        c=c.set_audio(AudioFileClip(mp3).volumex(1.0))
                        tmp.append(mp3)
                    except Exception: pass
                clips.append(c)

            final=concatenate_videoclips(clips, method='chain')
            final.write_videofile(mp4, fps=int(vid_fps.value), codec='libx264', audio_codec='aac')
            display(HTML('<div style="color:green;">影片完成！</div>'))
            if HAVE_FD:
                with open(mp4,'rb') as f:
                    display(FileDownload(data=f.read(), filename=os.path.basename(mp4), description='下載影片 (MP4)'))
            display(HTML(f'<a href="files/{mp4}" download>或點此下載 (MP4)</a>'))
            try:
                with open(mp4,'rb') as f: b64=base64.b64encode(f.read()).decode()
                display(HTML(f'<details><summary>若無法下載，改用備援</summary><a download="{os.path.basename(mp4)}" href="data:video/mp4;base64,{b64}">備援下載 (MP4)</a></details>'))
            except Exception: pass
        except Exception as e:
            display(HTML(f"<div style='color:red;'>製作影片失敗：{e}</div>"))

btn_doc.on_click(on_click_doc)
btn_vid.on_click(on_click_video)
